In [ ]:
# 정해진 예시에 따라 프롬프트 양식을 구현할 수 있음
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=1, 
    model_name = "gpt-3.5-turbo",
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler(),],)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
] 

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography and curture expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Korea"})



In [17]:
# 예제를 선택하는 기능 : BaseExampleSelector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate 
from langchain.prompts.example_selector.base import BaseExampleSelector 


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")


example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")


'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [ ]:
# Serialization and Composition
# proompt json 파일과 yaml 파일을 생성하고 load_prompt를 통해서 연결할 수 있음 
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate # 많은 프롬프트를 합칠 수 있음

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
    {example}
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

In [23]:
# Caching 에 관하여! - 1  + debug + SQLiteCache를 통한 db 생성으로 답변 가져옴
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make Korean pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Korean pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [4.74s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Korean pasta, also known as \"Korean spaghetti,\" is a popular fusion dish that combines traditional Korean flavors with Italian pasta. Here is a simple recipe to make Korean pasta:\n\nIngredients:\n- 8 oz spaghetti noodles\n- 1/2 lb ground beef or pork\n- 1 small onion, diced\n- 3 cloves garlic, minced\n- 1/4 cup soy sauce\n- 2 tbsp gochujang (Korean red chili paste)\n- 1 tbsp sugar\n- 1 tbsp sesame oil\n- 1 tbsp vegetable oil\n- Salt and pepper to taste\n- Optional toppings: sliced green onions, sesame seeds, shredded cheese\n\nInstructions:\n1. Cook the spaghetti noodles according to package instructions. Drain and set aside.\n2. In a large skillet, heat vegetable oil over medium heat. Add the diced onion and minced garlic, and sauté until frag

'Korean pasta, also known as "Korean spaghetti," is a popular fusion dish that combines traditional Korean flavors with Italian pasta. Here is a simple recipe to make Korean pasta:\n\nIngredients:\n- 8 oz spaghetti noodles\n- 1/2 lb ground beef or pork\n- 1 small onion, diced\n- 3 cloves garlic, minced\n- 1/4 cup soy sauce\n- 2 tbsp gochujang (Korean red chili paste)\n- 1 tbsp sugar\n- 1 tbsp sesame oil\n- 1 tbsp vegetable oil\n- Salt and pepper to taste\n- Optional toppings: sliced green onions, sesame seeds, shredded cheese\n\nInstructions:\n1. Cook the spaghetti noodles according to package instructions. Drain and set aside.\n2. In a large skillet, heat vegetable oil over medium heat. Add the diced onion and minced garlic, and sauté until fragrant.\n3. Add the ground beef or pork to the skillet and cook until browned and cooked through.\n4. In a small bowl, mix together soy sauce, gochujang, sugar, and sesame oil. Pour the sauce over the meat mixture and stir to combine.\n5. Add the

In [24]:
# Caching 에 관하여! - 2
chat.predict("How do you make Korean pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Korean pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Korean pasta, also known as \"Korean spaghetti,\" is a popular fusion dish that combines traditional Korean flavors with Italian pasta. Here is a simple recipe to make Korean pasta:\n\nIngredients:\n- 8 oz spaghetti noodles\n- 1/2 lb ground beef or pork\n- 1 small onion, diced\n- 3 cloves garlic, minced\n- 1/4 cup soy sauce\n- 2 tbsp gochujang (Korean red chili paste)\n- 1 tbsp sugar\n- 1 tbsp sesame oil\n- 1 tbsp vegetable oil\n- Salt and pepper to taste\n- Optional toppings: sliced green onions, sesame seeds, shredded cheese\n\nInstructions:\n1. Cook the spaghetti noodles according to package instructions. Drain and set aside.\n2. In a large skillet, heat vegetable oil over medium heat. Add the diced onion and minced garlic, and sauté until fragrant.

'Korean pasta, also known as "Korean spaghetti," is a popular fusion dish that combines traditional Korean flavors with Italian pasta. Here is a simple recipe to make Korean pasta:\n\nIngredients:\n- 8 oz spaghetti noodles\n- 1/2 lb ground beef or pork\n- 1 small onion, diced\n- 3 cloves garlic, minced\n- 1/4 cup soy sauce\n- 2 tbsp gochujang (Korean red chili paste)\n- 1 tbsp sugar\n- 1 tbsp sesame oil\n- 1 tbsp vegetable oil\n- Salt and pepper to taste\n- Optional toppings: sliced green onions, sesame seeds, shredded cheese\n\nInstructions:\n1. Cook the spaghetti noodles according to package instructions. Drain and set aside.\n2. In a large skillet, heat vegetable oil over medium heat. Add the diced onion and minced garlic, and sauté until fragrant.\n3. Add the ground beef or pork to the skillet and cook until browned and cooked through.\n4. In a small bowl, mix together soy sauce, gochujang, sugar, and sesame oil. Pour the sauce over the meat mixture and stir to combine.\n5. Add the

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("소주를 만드는 방법은?")
    b = chat.predict("맥주를 만드는 방법은?")
    print(a,b, "\n")
    print(usage.total_cost)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 소주를 만드는 방법은?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. 보리를 발효시킨 후 증류하여 술을 얻는다.\n2. 술을 숙성시키고 여러 번 걸러서 맑은 술을 얻는다.\n3. 맑은 술을 물과 섞어 알코올 도수를 조절한다.\n4. 필요에 따라 향신료나 과일 등을 첨가하여 맛을 조절한다.\n5. 마지막으로 여러 번 거름을 거쳐 깨끗한 상태로 마시기 좋은 소주가 완성된다.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "1. 보리를 발효시킨 후 증류하여 술을 얻는다.\n2. 술을 숙성시키고 여러 번 걸러서 맑은 술을 얻는다.\n3. 맑은 술을 물과 섞어 알코올 도수를 조절한다.\n4. 필요에 따라 향신료나 과일 등을 첨가하여 맛을 조절한다.\n5. 마지막으로 여러 번 거름을 거쳐 깨끗한 상태로 마시기 좋은 소주가 완성된다.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ]

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = ChatOpenAI(
    temperature = 0.1,
    model = "gpt-3.5-turbo",
    max_tokens = 400,
)

chat.save("model.jason")


AttributeError: 'ChatOpenAI' object has no attribute 'save'